In [28]:
import tweepy
from GetOldTweets import got

consumer_keys = ["51MI8RrYmzO4btCKG4Qb5uqAa", "glUJXqMWks3Hlu1QgPa2KvMHA"]
consumer_secrets = ["ajpPv3Ag0NvMEQLBIwiPyDyU78BbLZn8IS1gTba4x9ZOHNPMNM", "lDjQ3F9OraljWas6koMqYKzP9fFu6wdNiBEoBbvQUkNmr304Q1"]
access_tokens = ["3004471069-VDbNpT9NO0QOtiqKZXkoH5Flv4MArCflIYImXjn", "905545734748905472-1427NPHJ3VOgSxhygcK6gVNFTZgu4AE"]
access_tokens_secret = ["sP6KMjPZXxYAnaae8bOiauLjCVnx8bzWkBk4KU1iZBxdl", "vFrkF37p0B4C4cYiwMWhAHCiKj2r0wCrKcfutaWzg79fB"]

consumer_key = consumer_keys[1]
consumer_secret = consumer_secrets[1]
access_token = access_tokens[1]
access_token_secret = access_tokens_secret[1]

# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 



In [29]:
def printTweets(tweets):
    print("# tweets is: "+str(len(tweets)))
    counter = 0
    for tweet in tweets:
        print(str(counter)+": " + tweet.text)


        counter += 1

In [30]:
def printDateText(tweets):
    print("# tweets is: "+str(len(tweets)))
    counter = 0
    for tweet in tweets:
        print(str(counter)+": " + str(tweet.date)+"  ||  " + tweet.text)



        counter += 1

In [31]:
def getSortCriteria(tweet):
    print(tweet)
    return tweet.date



In [32]:
def writeToFile(path, tweets, language=None):
    output = open("tweettext.txt", "wb")
    counter = 0

    for tweet in new_tweets[:n]:
        # printing the text stored inside the tweet object
    #    print(tweet.user.screen_name,"Tweeted:",tweet.text)
        if (language == None) or (language == tweet.lang):
            adjusted_tweet = (tweet.text).encode('utf-8')

            output.write(adjusted_tweet)
            print(str(counter)+adjusted_tweet)
            output.write("\n")

            print(tweet.created_at)

            counter += 1
            
  
    output.close()

In [33]:
import urllib2
from bs4 import BeautifulSoup as bs

#https://finance.yahoo.com/quote/AMZN
    
def get_historical_data(name, number_of_days):
    data = []
    url = "https://finance.yahoo.com/quote/" + name + "/history?period1=946713600&period2=1510732800&interval=1d&filter=history&frequency=1d/"
    rows = bs(urllib2.urlopen(url).read()).findAll('table')[0].tbody.findAll('tr')

    for each_row in rows:
        divs = each_row.findAll('td')
        if divs[1].span.text  != 'Dividend': #Ignore this row in the table
            #I'm only interested in 'Open' price; For other values, play with divs[1 - 5]
            data.append({'Date': divs[0].span.text, 'Open': float(divs[1].span.text.replace(',',''))})

    return data[number_of_days:number_of_days+1]


In [34]:
import csv


def getPriceFromCSV(filepath, date):
    with open(filepath, 'rt') as f:
        #skip header
        next(f)
        
        reader = csv.reader(f)
        rownum = 1
        for row in reader: 
            rownum += 1

            
            #CSV appears to be in mm/dd/yyyy format but parsing is yyyy/mm/dd ? weird
            adjusted_date = (dt.datetime.strptime(str(row[0]), '%Y-%m-%d')).date()

            if adjusted_date == date:
                print("price of interest is: ")
                print((str(row[1]), str(row[2]), str(row[3]), str(row[4]), str(row[5])) )
                return [str(row[1]), str(row[2]), str(row[3]), str(row[4]), str(row[5])]
    

In [35]:
#returns filtered tweets
def filterTweets(tweets, remove_duplicates=True):
    combined_tweets = tweets
    if remove_duplicates: 
        combined_tweets = list(set(combined_tweets)) #remove duplicates
        
    counter = 0
    for tweet in combined_tweets:
    #    print(counter)
        try:
            if tweet.date:
                counter += 1
                continue
        except AttributeError:
            print("exception on number: " + str(counter))
            print(tweet.text)
           # combined_tweets.remove(tweet)
            combined_tweets = list(filter(lambda twt: twt != tweet, combined_tweets))

            counter += 1

    print("filtered: " + str(counter) )      
#    printDateText(combined_tweets)
    return combined_tweets

In [41]:
from yahoo_finance import Share
import datetime as dt
from datetime import timedelta
import os

remove_duplicates = True
stock_ticker = "AMZN"
query = "AMZN"
max_tweets = 10000
n = max_tweets

#twitter was founded in 2006. Start in 2007 for safety
base_date = "2007-01-03"
start_date = "2007-01-04"
end_date = "2017-10-31"

#new_tweets = [tweet for tweet in tweepy.Cursor(api.search, 
                                   #        q=query).items(max_tweets)]

print("Pulling tweets...")    
#get old tweets

base_date_dt = dt.datetime.strptime(base_date, '%Y-%m-%d')
start_date_dt = dt.datetime.strptime(start_date, '%Y-%m-%d')
end_date_dt = dt.datetime.strptime(end_date, '%Y-%m-%d')
delta = end_date_dt - start_date_dt

num_days = int(delta.days)
print(num_days)
begin = base_date_dt.strftime('%Y-%m-%d')

i = 1
tweets = []
processed_tweets = []

path = os.getcwd()+"/"
f = open(path+'tweets_'+stock_ticker+'.txt', 'a')    
f2 = open(path+'labels_'+stock_ticker+'.txt', 'a')
#perform data structure swapping and clearing to save memory for large runs
for single_date in (start_date_dt + timedelta(n) for n in range(num_days-1)):
    date_only = single_date.strftime('%Y-%m-%d')
    
    end = single_date.strftime('%Y-%m-%d')

    tweetCriteria = got.manager.TweetCriteria().setQuerySearch(query).setSince(begin).setUntil(end).setMaxTweets(max_tweets)
    tweets.extend(got.manager.TweetManager.getTweets(tweetCriteria))#[0]
    
    printTweets(tweets)
    print("processing day: " + begin)
    print("tweets size: " + str(len(tweets)))
    
    #filter the sets of tweets in batches of 20
    if ((i % 20) == 0) or ((num_days - i) < 20):
        processed_tweets.extend(filterTweets(tweets, remove_duplicates=True))
        tweets = []
        
        #sort by date
        processed_tweets.sort(key=getSortCriteria)
        #printDateText(combined_tweets)


        #find associated stock prices
        #stock = Share( stock_ticker )
        cur_date = dt.datetime.now()
        print(cur_date)

        tweet_labels = []
        cutoff_date = dt.datetime.strptime("2017-10-30", '%Y-%m-%d')


        datapath = os.getcwd()+"/StockData/"+stock_ticker+".csv"
        cutoff_threshold = timedelta(days=0)

        for tweet in processed_tweets:
            tweet_date = tweet.date
            print("tweet date is: " + str(tweet_date) +" of type: " + str(type(tweet_date)))

            #pull from file since the tweet is too old
           # if (tweet_date - cutoff_date) < cutoff_threshold:
            tweet_dateOnly = (tweet_date).date()
            price = getPriceFromCSV(datapath, tweet_dateOnly)
            if(price == None): #could have been a holiday
                #processed_tweets.remove(tweet)
                processed_tweets = list(filter(lambda twt: twt != tweet, processed_tweets))


            else:
                tweet_labels.append(price)
                #            print("type is: " + str(type(open_price)))

            """
                    
            else:
                delta = cur_date - tweet_date
                delta_days = delta.days
                print("delta days: "+str(delta_days))

                price = get_historical_data(stock_ticker, delta_days)
                tweet_labels.extend(str(price))
                """
        #test
        #print(get_historical_data(stock_ticker, 10))


        #print(counter)    
        #printDateText(processed_tweets)

        #dump to file
        for t in processed_tweets:
            f.write(((t.date).strftime('%Y-%m-%d')) + ",  " + (t.text).encode('utf-8'))
            f.write('\n')

        for price in tweet_labels:
            print("iterating through price")
            print(price)
#            print(str(price[0])+',' + str(price[1])+',' + str(price[2])+',' + str(price[3])+',' + str(price[4]))
            f2.write(str(price[0])+',' + str(price[1])+',' + str(price[2])+',' + str(price[3])+',' + str(price[4]))
            f2.write('\n')

        processed_tweets = []


#    printTweets(old_tweets)
    
    #set new begin to be the current day for the next iteration
    begin = end
    
    i+=1

f.close()
f2.close()
      


lines_in_tweets = 0
f = open(path+'tweets_'+stock_ticker+'.txt', 'r')
for line in f:
    lines_in_tweets += 1
    
f.close()

lines_in_labels = 0
f2 = open(path+'labels_'+stock_ticker+'.txt', 'r')
for line in f2:
    lines_in_labels += 1
    
print("#lines text: " + str(lines_in_tweets) +"  #lines labels: " + str(lines_in_labels))


Pulling tweets...
3953
# tweets is: 0
processing day: 2007-01-03
tweets size: 0
# tweets is: 0
processing day: 2007-01-04
tweets size: 0
# tweets is: 0
processing day: 2007-01-05
tweets size: 0
# tweets is: 0
processing day: 2007-01-06
tweets size: 0
# tweets is: 0
processing day: 2007-01-07
tweets size: 0
Twitter weird response. Try to see on browser: https://twitter.com/search?q=%20AMZN%20since%3A2007-01-08%20until%3A2007-01-09&src=typd


SystemExit: 